In [1]:
%pip install duckduckgo_search requests
import os
import requests
from duckduckgo_search import DDGS

# --- Configuration ---
# The search query you want to use
SEARCH_QUERY = "Gothic Revival architecture house"

# The directory where you want to save the images
DOWNLOAD_DIRECTORY = "gothic_revival_images"

# The maximum number of images you want to download
MAX_IMAGES = 1

def download_images():
    """
    Searches for images using DuckDuckGo and downloads them.
    """
    print(f"Searching for '{SEARCH_QUERY}' on DuckDuckGo...")

    # Use a context manager for the DDGS client
    with DDGS() as ddgs:
        # The ddgs.images() function is a generator that yields search results
        # We specify the query, region, safesearch, and max_results
        search_results = ddgs.images(
            keywords=SEARCH_QUERY,
            region="us-en",      # United States, English
            safesearch="off",    # or "on", "moderate"
            size=None,           # Any size
            color=None,          # Any color
            type_image=None,     # Any type (photo, clipart, etc.)
            layout=None,         # Any layout (square, tall, wide)
            license_image=None,  # Any license
            max_results=MAX_IMAGES
        )

        print(f"Found image results. Starting download of up to {MAX_IMAGES} images...")

        # Create the download directory if it doesn't exist
        if not os.path.exists(DOWNLOAD_DIRECTORY):
            os.makedirs(DOWNLOAD_DIRECTORY)
            print(f"Created directory: {DOWNLOAD_DIRECTORY}")

        # Loop through the search results to download images
        for i, result in enumerate(search_results):
            image_url = result.get("image")
            if not image_url:
                print(f"Skipping result {i+1}: No image URL found.")
                continue

            try:
                # Get the image content from the URL
                print(f"[{i+1}/{MAX_IMAGES}] Downloading: {image_url[:70]}...")
                response = requests.get(image_url, timeout=10)
                
                # Raise an exception if the request was unsuccessful (e.g., 404 Not Found)
                response.raise_for_status()

                # Determine the file extension (default to .jpg)
                file_extension = os.path.splitext(image_url)[1]
                if not file_extension or len(file_extension) > 5: # Basic check for valid extension
                    file_extension = ".jpg"
                
                # Create a unique filename for the image
                filename = f"gothic_revival_{i+1}{file_extension}"
                filepath = os.path.join(DOWNLOAD_DIRECTORY, filename)

                # Save the image to the directory
                with open(filepath, 'wb') as f:
                    f.write(response.content)
                
                print(f"   -> Saved as {filepath}")

            except requests.exceptions.RequestException as e:
                print(f"   -> Error downloading image {i+1}: {e}")
            except Exception as e:
                print(f"   -> An unexpected error occurred for image {i+1}: {e}")

    print("\nDownload process finished.")

if __name__ == "__main__":
    download_images()

  Using cached duckduckgo_search-8.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached primp-0.15.0-cp38-abi3-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached charset_normalizer-3.4.3-cp313-cp313-macosx_10_13_universal2.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached duckduckgo_search-8.1.1-py3-none-any.whl (18 kB)
Using cached charset_normalizer-3.4.3-cp313-cp313-macosx_10_13_universal2.whl (205 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached click-8.2.1-py3-none-any.whl (102 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.7 MB/s  0:00:00m0:00:01
Using cached primp-0.15.0-cp38-abi3-macosx_11_0_arm64.whl (3.0

/var/folders/s8/6lj08kz17ls03rcr6ms85hc00000gn/T/ipykernel_46601/2650911059.py:23: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Found image results. Starting download of up to 1 images...
Created directory: gothic_revival_images
[1/1] Downloading: https://i.pinimg.com/originals/89/dd/bf/89ddbfffe93a3ed8d70f729422ca43...
   -> Saved as gothic_revival_images/gothic_revival_1.jpg

Download process finished.


In [7]:
import os
import re
import requests
from duckduckgo_search import DDGS
import time

# --- Configuration ---

# List of house styles transcribed from the image
HOUSE_STYLES = [
    "Saltbox", "Georgian", "Federal", "Greek Revival", "Gothic Revival",
    "Italianate", "Second Empire", "Queen Anne", "Shingle",
    "Richardsonian Romanesque", "Folk Victorian", "Colonial Revival",
    "Cape Cod", "Neo Classical", "Tudor Revival", "Spanish Colonial Revival",
    "French Revival", "Pueblo Revival", "Craftsman", "Modernistic",
    "International", "Ranch/Prairie"
]

# The base directory where all style folders will be created
BASE_DOWNLOAD_DIRECTORY = "architectural_style_images"

# The maximum number of images to download for each style
MAX_IMAGES_PER_STYLE = 50

def sanitize_foldername(name):
    """
    Cleans a string to be used as a valid folder name.
    Replaces spaces and slashes with underscores and removes invalid characters.
    """
    name = name.replace('/', '_').replace(' ', '_')
    name = re.sub(r'[^\w-]', '', name)
    return name

def download_images_for_style(style, base_dir, max_images):
    """
    Searches for images of a given architectural style and downloads them
    into a dedicated sub-folder.
    """
    print(f"\n{'='*20}\nProcessing style: {style}\n{'='*20}")

    style_folder_name = sanitize_foldername(style)
    download_path = os.path.join(base_dir, style_folder_name)
    os.makedirs(download_path, exist_ok=True)
    print(f"Saving images to: {download_path}")

    search_query = f"{style} house architecture"
    print(f"Searching for: '{search_query}'...")

    # --- FIX IS HERE ---
    # Initialize the counter BEFORE the try block to ensure it always exists.
    download_count = 0

    try:
        with DDGS() as ddgs:
            search_results = ddgs.images(
                keywords=search_query,
                region="us-en",
                safesearch="off",
                max_results=max_images
            )
            
            for i, result in enumerate(search_results):
                # The loop will only run if search_results is not empty
                image_url = result.get("image")
                if not image_url:
                    print(f"Skipping result {i+1}: No image URL found.")
                    continue

                try:
                    print(f"[{i+1}/{max_images}] Downloading: {image_url[:70]}...")
                    response = requests.get(image_url, timeout=15)
                    response.raise_for_status()

                    file_ext = os.path.splitext(image_url)[1]
                    if not file_ext or len(file_ext) > 5:
                        content_type = response.headers.get('content-type')
                        if content_type and 'jpeg' in content_type: file_ext = '.jpg'
                        elif content_type and 'png' in content_type: file_ext = '.png'
                        else: file_ext = '.jpg'

                    filename = f"{style_folder_name}_{download_count + 1}{file_ext}"
                    filepath = os.path.join(download_path, filename)

                    with open(filepath, 'wb') as f:
                        f.write(response.content)
                    
                    print(f"   -> Saved as {filepath}")
                    download_count += 1

                except requests.exceptions.RequestException as e:
                    print(f"   -> Error downloading image {i+1}: {e}")
                except Exception as e:
                    print(f"   -> An unexpected error occurred for image {i+1}: {e}")

    except Exception as e:
        print(f"A critical error occurred while searching for {style}: {e}")
    
    # This print statement is now safe, as download_count is guaranteed to exist.
    print(f"\nFinished processing '{style}'. Downloaded {download_count} images.")


def main():
    """
    Main function to loop through all house styles and initiate downloads.
    """
    os.makedirs(BASE_DOWNLOAD_DIRECTORY, exist_ok=True)
    
    total_styles = len(HOUSE_STYLES)
    for index, style in enumerate(HOUSE_STYLES):
        print(f"\n--- Starting style {index + 1} of {total_styles} ---")
        download_images_for_style(
            style=style,
            base_dir=BASE_DOWNLOAD_DIRECTORY,
            max_images=MAX_IMAGES_PER_STYLE
        )
        time.sleep(2)
        
    print("\n\nAll architectural styles have been processed. Scraping complete.")


if __name__ == "__main__":
    main()


--- Starting style 1 of 22 ---

Processing style: Saltbox
Saving images to: architectural_style_images/Saltbox
Searching for: 'Saltbox house architecture'...


/var/folders/s8/6lj08kz17ls03rcr6ms85hc00000gn/T/ipykernel_46601/2218560444.py:54: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


A critical error occurred while searching for Saltbox: https://duckduckgo.com/i.js?o=json&q=Saltbox+house+architecture&l=us-en&vqd=4-77971580884481321998838787339606919092&p=-1&f=%2C%2C%2C%2C%2C 403 Ratelimit

Finished processing 'Saltbox'. Downloaded 0 images.

--- Starting style 2 of 22 ---

Processing style: Georgian
Saving images to: architectural_style_images/Georgian
Searching for: 'Georgian house architecture'...


/var/folders/s8/6lj08kz17ls03rcr6ms85hc00000gn/T/ipykernel_46601/2218560444.py:54: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


A critical error occurred while searching for Georgian: https://duckduckgo.com/i.js?o=json&q=Georgian+house+architecture&l=us-en&vqd=4-304468755422605327266758401698801492877&p=-1&f=%2C%2C%2C%2C%2C 403 Ratelimit

Finished processing 'Georgian'. Downloaded 0 images.

--- Starting style 3 of 22 ---

Processing style: Federal
Saving images to: architectural_style_images/Federal
Searching for: 'Federal house architecture'...


...

A critical error occurred while searching for Modernistic: https://duckduckgo.com/i.js?o=json&q=Modernistic+house+architecture&l=us-en&vqd=4-113977532651230231170303328793733369058&p=-1&f=%2C%2C%2C%2C%2C 403 Ratelimit

Finished processing 'Modernistic'. Downloaded 0 images.

--- Starting style 21 of 22 ---

Processing style: International
Saving images to: architectural_style_images/International
Searching for: 'International house architecture'...


/var/folders/s8/6lj08kz17ls03rcr6ms85hc00000gn/T/ipykernel_46601/2218560444.py:54: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


A critical error occurred while searching for International: https://duckduckgo.com/i.js?o=json&q=International+house+architecture&l=us-en&vqd=4-153296329911244471734284389022245235945&p=-1&f=%2C%2C%2C%2C%2C 403 Ratelimit

Finished processing 'International'. Downloaded 0 images.

--- Starting style 22 of 22 ---

Processing style: Ranch/Prairie
Saving images to: architectural_style_images/Ranch_Prairie
Searching for: 'Ranch/Prairie house architecture'...


/var/folders/s8/6lj08kz17ls03rcr6ms85hc00000gn/T/ipykernel_46601/2218560444.py:54: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


A critical error occurred while searching for Ranch/Prairie: https://duckduckgo.com/i.js?o=json&q=Ranch%2FPrairie+house+architecture&l=us-en&vqd=4-331094878150244586187329362068455889885&p=-1&f=%2C%2C%2C%2C%2C 403 Ratelimit

Finished processing 'Ranch/Prairie'. Downloaded 0 images.


All architectural styles have been processed. Scraping complete.
